In [ ]:
from __future__ import division

import pandas as pd
import warnings
import colorsys

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_json("../../data/parade/blocks-colors.json")

# Colors only

Only one occurence counted per color in the viz.  

- keep the `colors` column only
- flatten the object to one occurence per color
- group by colors / sum of occurences

In [ ]:
df_colors = df[["colors"]]
df_colors.colors = df_colors.colors.astype(str)
df_colors.colors = df_colors.colors.str.replace("{","").str.replace("}","")
df_colors = df_colors.apply(lambda x: x.str.split(',').explode()).reset_index(drop=True)

In [ ]:
df_colors.colors = df_colors.colors.str.replace("\'","")
df_colors[["colors_def","B"]] = df_colors["colors"].str.split(":",expand=True)
df_colors = df_colors[["colors_def","colors"]]
df_colors = df_colors.groupby("colors_def")["colors"].count().reset_index()
df_colors.colors_def = df_colors["colors_def"].str.replace(" ","")

In [ ]:
def convert_rgb_to_hls(r, g, b):
    h, l, s = colorsys.rgb_to_hls(r/255, g/255, b/255)
    return "(" + str(int(round(h * 360))) + ", " + str(int(round(l * 100))) + ", " + str(int(round(s * 100))) + ")"

In [ ]:
def converting_HEX_RGB(arg):
    h = arg.lstrip('#')
    return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

In [ ]:
def converting_RGB_HLS(arg):
    R = arg[0]
    G = arg[1]
    B = arg[2]
    return convert_rgb_to_hls(R, G, B)

In [ ]:
df_colors["RGB"] = df_colors["colors_def"].apply(converting_HEX_RGB)
df_colors["HLS"] = df_colors["RGB"].apply(converting_RGB_HLS)
df_colors_min = df_colors[["HLS","colors"]]

In [ ]:
df_colors_min["HLS"] = df_colors_min["HLS"].str.replace("(","").str.replace(")","")

In [ ]:
df_colors_min[["H","L","S"]] = df_colors_min["HLS"].str.split(",", expand=True)

df_colors_min["H"] = df_colors_min["H"].astype(int)
df_colors_min["S"] = df_colors_min["S"].astype(int)
df_colors_min["L"] = df_colors_min["L"].astype(int)

In [ ]:
del df_colors_min["HLS"]

## Calling function

In [ ]:
def sample(size):
    df = df_colors_min.sort_values("colors",ascending=False).head(size)
    
    # adding rank on Hue / Saturation / Lightness
    df.sort_values("H",inplace=True)
    df.reset_index(inplace=True, drop=True)
    df["hue_rank"] = df.index

    df.sort_values("S",inplace=True)
    df.reset_index(inplace=True, drop=True)
    df["saturation_rank"] = df.index

    df.sort_values("L",inplace=True)
    df.reset_index(inplace=True, drop=True)
    df["lightness_rank"] = df.index

    df.sort_values("colors",ascending=False,inplace=True)
    df = df.reset_index(drop=True)

    df.to_csv("../../data/d3_parade/colors-rank-{}.csv".format(size))

    return df

In [ ]:
df_1000 = sample(1000)